In [134]:
import pandas as pd
from dateutil import parser
from datetime import datetime, timedelta
import numpy as np
#primeDF = pd.read_csv(f"itemDataFrames/ash_prime_set.csv", index_col=0)
#print(parser.parse("2024-01-19 02:17:31") - parser.parse("2022-01-19 02:17:31"))

In [130]:
#From https://stackoverflow.com/questions/46030055/python-time-weighted-average-pandas-grouped-by-time-interval
#Answer by hugovdberg
def resample_time_weighted_mean(x, target_index, closed=None, label=None):
    shift = 1 if closed == "right" else -1
    fill = "bfill" if closed == "right" else "ffill"
    # Determine length of each interval (daylight saving aware)
    extended_index = target_index.union(
        [target_index[0] - target_index.freq, target_index[-1] + target_index.freq]
    )
    interval_lengths = -extended_index.to_series().diff(periods=shift)

    # Create a combined index of the source index and target index and reindex to combined index
    combined_index = x.index.union(extended_index)
    x = x.reindex(index=combined_index, method=fill)
    interval_lengths = interval_lengths.reindex(index=combined_index, method=fill)

    # Determine weights of each value and multiply source values
    weights = -x.index.to_series().diff(periods=shift) / interval_lengths
    x = x.mul(weights, axis=0)

    # Resample to new index, the final reindex is necessary because resample 
    # might return more rows based on the frequency
    return (
        x.resample(target_index.freq, closed=closed, label=label)
        .sum()
        .reindex(target_index)
    )

In [131]:

#Returns a dataframe object with datetime column converted to datetime objects
#itemName has underscores such as ash_prime_set
def initDF(itemName):
    df = pd.read_csv(f"itemDataFrames/{itemName}.csv", index_col=0)
    df["datetime"] = pd.to_datetime(df['datetime'])
    return df

#converts a dataframe to the timeWeightedAverage dataframe
#timestep is a string of the form "15min" or "30T"
def convertTWA(df, timeStep):
    df = df.set_index("datetime")
    opts = dict(closed="right", label="right")
    return resample_time_weighted_mean(
        df, pd.DatetimeIndex(df.resample(timeStep, **opts).groups.keys(), freq="infer"), **opts
        ).reset_index(names="datetime")

#initializes from scratch a time weighted average dataframe according to
#a timestep to group by - timestep is a string of the form "15min" or "30T"
def initGroupedDF(itemName, timeStep):
    return df

def clearMissingData(originalDF, TWADF):
    filteredTWA = TWADF.copy()
    for i in TWADF.index[:-1]:
        if len(originalDF[(originalDF['datetime'] >= TWADF['datetime'].iloc[i]) & (originalDF['datetime'] < TWADF['datetime'].iloc[i+1])]) == 0:
            filteredTWA.iloc[i, [x for x in range(1, filteredTWA.shape[1])]] = None
    return filteredTWA

%run PrimeItemParser.ipynb
#for name in primeSetNames:
    #timeStep = "15min"
    #df = initDF(name)
    #TWAdf = convertTWA(df, timeStep)
    #TWAdf = TWAdf.iloc[0:-1]
    #filteredDF = clearMissingData(df, TWAdf)
    #print(name)
    #display(filteredDF)

In [132]:
import ipywidgets as widgets
from IPython.display import display
x = widgets.Combobox(
    # value='John',
    placeholder='Choose A Warframe',
    options=primeSetNames,
    description='Item Name:',
    ensure_option=True,
    disabled=False,
    style=dict(description_width='initial')
)
y = widgets.Text(
    value='15',
    placeholder='15',
    description='Time Interval:',
    disabled=False,
    style=dict(description_width='initial')
)
z = widgets.Dropdown(
    options=[("Seconds", 's'), ("Minutes", 'min'), ("Hours", 'h')],
    value='min',
    description='Time Unit:',
    disabled=False,
    style=dict(description_width='initial')
)
timeOpDic = {"s" : "second", "min": "minute", "h": "hour"}

display(x)
display(y)
display(z)

Combobox(value='', description='Item Name:', ensure_option=True, options=('ash_prime_set', 'atlas_prime_set', …

Text(value='15', description='Time Interval:', placeholder='15', style=TextStyle(description_width='initial'))

Dropdown(description='Time Unit:', index=1, options=(('Seconds', 's'), ('Minutes', 'min'), ('Hours', 'h')), st…

In [133]:
from IPython.display import clear_output
button = widgets.Button(description="Show DataFrame!")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        clear_output(wait=True)
        timeStep = y.value + z.value
        try:
            df = initDF(x.value)
            TWAdf = convertTWA(df, timeStep)
            TWAdf = TWAdf.iloc[0:-1]
            filteredDF = clearMissingData(df, TWAdf)
            print(f"Displaying last 20 rows of {x.value.replace('_', ' ').title()}\nGrouped by {y.value} {timeOpDic[z.value]} intervals.")
            display(filteredDF[-20:])
            print("*NaN Values imply no data was collected during the time window.")
        except FileNotFoundError:
            print("Enter A Valid Warframe Name")

button.on_click(on_button_clicked)

Button(description='Show DataFrame!', style=ButtonStyle())

Output()